Side loading data inputs method 1

In [2]:
import pywapor
import xarray as xr
from functools import partial
import pywapor.se_root as se_root

et_look_config = pywapor.general.levels.pre_et_look_levels(level = "level_1")
se_root_config = pywapor.general.levels.pre_se_root_levels(level = "level_1")

/opt/conda/envs/pywapor/lib/python3.11/site-packages/pywapor/enhancers/smooth/core.py:21: NumbaExperimentalFeatureWarning: Use of isinstance() detected. This is an experimental feature.
  v = np.where(Y > Z, a_, 1 - a_)
/opt/conda/envs/pywapor/lib/python3.11/site-packages/pywapor/enhancers/smooth/core.py:128: NumbaExperimentalFeatureWarning: Use of isinstance() detected. This is an experimental feature.
  Y_ = np.where(w == 0, 0, Y)


Create a list of the meteorological variables.

In [1]:
# meteo_vars = ['t_air', 't_air_max', 't_air_min', 'u2m', 'v2m', 'qv', 'p_air', 'p_air_0', 'wv']
meteo_vars = ['t_air_min']

In [3]:
et_look_config["t_air_min"]

{'products': [{'source': 'GEOS5',
   'product_name': 'inst3_2d_asm_Nx',
   'enhancers': 'default'}],
 'composite_type': 'min',
 'temporal_interp': 'linear',
 'spatial_interp': 'bilinear'}

We would have to convert the geotiff files to .cn format and locate them

In [4]:
def meteo_sideload(**kwargs):
    fh = r"/workspaces/GIT716.ET/NetCDF_files/airTemp_min.nc"
    ds = xr.open_dataset(fh)
    return ds

In [6]:
meteo_config = [{"source": meteo_sideload, "product_name": "SAWS_data", "enhancers": []}]
for var in meteo_vars:
    et_look_config[var]["products"] = meteo_config
    se_root_config[var]["products"] = meteo_config

In [7]:
et_look_config["t_air_min"]

{'products': [{'source': <function __main__.meteo_sideload(**kwargs)>,
   'product_name': 'SAWS_data',
   'enhancers': []}],
 'composite_type': 'min',
 'temporal_interp': 'linear',
 'spatial_interp': 'bilinear'}

In [8]:
et_look_config["se_root"]

{'products': [{'source': functools.partial(<function se_root at 0x7f8c050c6660>, bin_length=1, sources='level_1'),
   'product_name': 'v2',
   'enhancers': 'default'}],
 'composite_type': 'max',
 'temporal_interp': None,
 'spatial_interp': 'bilinear'}

In [9]:
se_root_dler = partial(se_root.se_root, sources = se_root_config)
et_look_config["se_root"]["products"][0]["source"] = se_root_dler

In [10]:
et_look_config["se_root"]

{'products': [{'source': functools.partial(<function se_root at 0x7f8c050c6660>, sources={'ndvi': {'products': [{'source': 'MODIS', 'product_name': 'MOD13Q1.061', 'enhancers': 'default', 'is_example': True}, {'source': 'MODIS', 'product_name': 'MYD13Q1.061', 'enhancers': 'default'}], 'temporal_interp': 'linear', 'spatial_interp': 'nearest'}, 'lst': {'products': [{'source': 'MODIS', 'product_name': 'MOD11A1.061', 'enhancers': 'default'}, {'source': 'MODIS', 'product_name': 'MYD11A1.061', 'enhancers': 'default'}], 'temporal_interp': None, 'spatial_interp': 'nearest'}, 't_air': {'products': [{'source': 'GEOS5', 'product_name': 'inst3_2d_asm_Nx', 'enhancers': 'default'}], 'temporal_interp': 'linear', 'spatial_interp': 'bilinear'}, 't_air_max': {'products': [{'source': 'GEOS5', 'product_name': 'inst3_2d_asm_Nx', 'enhancers': 'default'}], 'temporal_interp': 'linear', 'spatial_interp': 'bilinear'}, 't_air_min': {'products': [{'source': <function meteo_sideload at 0x7f8c64156b60>, 'product_nam

In [11]:
folder = r"/workspaces/GIT716.ET/Results_level2"
timelim = ["2020-01-01", "2020-01-10"]
latlim = [-34.050015, -33.369369]
lonlim = [18.723629, 19.390929]
input_ds = pywapor.pre_et_look.main(folder, latlim, lonlim, timelim, sources = et_look_config)

> PRE_ET_LOOK
    --> Collecting `ndvi` from `MODIS.MOD13Q1.061`.
        --> Server error 500 Server Error: Internal Server Error for url: https://opendap.cr.usgs.gov/opendap/hyrax/oauth2?code=76259ec004123e1c449bdc059e02be7e8688aa116a8fe05e3b0f2e9b41960ad4&state=aHR0cHM6Ly9vcGVuZGFwLmNyLnVzZ3MuZ292L29wZW5kYXAvaHlyYXgvTU9EMTNRMS4wNjEvaDE5djEyLm5jbWwubmM0P1hEaW0lMkNZRGltJTJDdGltZQ [<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>500 Internal Server Error</title>
</head><body>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error or
misconfiguration and was unable to complete
your request.</p>
<p>Please contact the server administrator at 
 root@localhost to inform them of the time this error occurred,
 and the actions you performed just before this error.</p>
<p>More information about this error may be available
in the server error log.</p>
</body></html>
].
        --> Trying to download /tmp/tmpd1woxohc.nc, attempt 2 of 3 in 15 seconds.
   

KeyboardInterrupt: 